In [2]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 54 kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619434 sha256=cd4a6293a91a9945422cc245ede27a6bb445681890b91b759f1f3fc1bb9eec5e
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
from surprise import KNNWithMeans, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd
unames = ['userId', 'gender', 'age', 'occupation', 'zip']
users = pd.read_table('users.dat', sep='::', 
                      header=None, names=unames, engine='python')
rnames = ['userId', 'movieId', 'rating', 'timestamp']
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=rnames, engine='python')

mnames = ['movieId', 'title', 'genres']
movies = pd.read_table('movies.dat', sep='::', header=None, names=mnames, engine='python')

In [4]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

dataset_0 = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

from surprise.model_selection.validation import cross_validate

n = 1
dataset = dataset_0.sample(frac=1)[:int(n*len(dataset_0))].reset_index(drop=True)

reader = Reader(rating_scale=(dataset.rating.min(), dataset.rating.max()))
data = Dataset.load_from_df(dataset, reader)

int(n*len(dataset_0))

472550

### ^количество строк массива с данными

In [5]:
algo = KNNBaseline(k=40, min_k=20, sim_options={'name': 'pearson_baseline', 'user_based': True})
result = cross_validate(algo, data, ['rmse'], n_jobs = -1, verbose = True)

Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8720  0.8786  0.8806  0.8777  0.8764  0.8771  0.0029  
Fit time          22.73   24.17   25.73   24.74   16.55   22.78   3.27    
Test time         54.90   58.32   55.49   51.07   26.61   49.28   11.57   


### Лучший результат который удалось получить Mean_RMSE = 0.8771